# IMF API 2

The method presented in the part 1 provides access to data from the IMF API. Here in part 2, techniques for finding the right codes to make successful API requests are presented, using the requests package and Python 2.7. 

### Find the series list
The 'Dataflow' method offers JSON formatted information on which series are available through the API. To find the series of interest (for example Direction of Trade Statistics) with Python, we can search a dictionary with the series names and their IDs. 

In [1]:
import requests  # Python 2.7, requests version 2.12.4

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  # Method with series information
search_term = 'Trade'  # Term to find in series names
series_list = requests.get('{}{}'.format(url, key)).json()\
            ['Structure']['Dataflows']['Dataflow']
# Use dict keys to navigate through results:
for series in series_list:
    if search_term in series['Name']['#text']:
        print '{}: {}'.format(series['Name']['#text'],
                              series['KeyFamilyRef']['KeyFamilyID'])

Direction of Trade Statistics (DOTS): DOT
International Trade in Services: ITS


### Finding the dimensions of the series

The exact format of the key in our API request is determined by the structure of the series. Direction of Trade Statistics, which are grouped by importer and exporter pair rather than by country exemplifies the need to first determine series structure.

The dimensions of the data are found with the DataStructure method and series specific, so that the full key becomes 'DataStructure/DOT'

In [2]:
key = 'DataStructure/IFS'  # Method / series
dimension_list = requests.get('{}{}'.format(url, key)).json()\
            ['Structure']['KeyFamilies']['KeyFamily']\
            ['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print 'Dimension {}: {}'.format(n+1, dimension['@codelist'])

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_IFS
Dimension 3: CL_INDICATOR_IFS


In this case, the dimensions correspond to: 1) frequency, 2) country or reference area 1, 3) indicator (such as total exports), and 4) country or reference area 2. That is, the monthly value of goods exports from Italy to France would be M.IT.TXG_FOB.FR

### Finding the codes for each dimension

The codes which correspond to the dimensions identified above are combined, in order, and separated by periods, to complete the API request url. To find the list of possible codes for each dimension, we can use the CodeList method, shown below for dimension 1, frequency. 

In [3]:
# Example: codes for third dimension, which is 2 in python
key = 'CodeList/{}'.format(dimension_list[2]['@codelist'])
code_list = requests.get('{}{}'.format(url, key)).json()\
            ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list:
    print '{}: {}'.format(code['Description']['#text'], code['@value'])

Agricultural Production, Index: AAP_IX
All Commodities (Fuel and Non Fuel), Price, Index: PZPI_IX
All items, Capital City, Index: PCPI_A1_IX
Assets, Direct investment, Debt instruments, US Dollars: IADD_BP6_USD
Assets, Direct investment, Equity and investment fund shares , US Dollars: IADE_BP6_USD
Assets, Direct investment, US Dollars: IAD_BP6_USD
Assets, FDI Abroad (BPM5), US Dollars: IAD_USD
Assets, Financial derivatives (other than reserves) and employee stock options , US Dollars: IADF_BP6_USD
Assets, Other Investment (BPM5), US Dollars: IAO_USD
Assets, Other Investment, Banks (BPM5), US Dollars: IAOB_USD
Assets, Other Investment, General Government (BPM5), US Dollars: IAOG_USD
Assets, Other Investment, Monetary Authorities (BPM5), US Dollars: IAOA_USD
Assets, Other investment, Other equity , US Dollars: IAOE_BP6_USD
Assets, Other Investment, Other Sectors (BPM5), US Dollars: IAOOS_USD
Assets, Portfolio investment , US Dollars: IAP_BP6_USD
Assets, Portfolio Investment excluding Fin

### Variations and notes
Once a series has been identified, it can be a challenge to determine which combination of keys returns valid data. Often an indicator has data available at only one frequency--whichever frequency of compilation occurs in the source country, though there are exceptions. 

The number of indicators varies by series. In the case of International Financial Statistics (IFS), there are more than 2500 indicators, which there are four indicators in the Direction of Trade Statistics series. The same search technique used to find the series names can be used to filter IFS indicators.

Part 3 shows how to retrieve metadata and make a request with more than one reference area (country).